<a href="https://colab.research.google.com/github/Martinccv/Proyecto_final_DS/blob/main/Entrega3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3er entrega - Sistemas de recomendación

En esta entrega vamos a trabajar con un sistema de recomendación.

Vamos a aprovechar la competencia de Telecom que vimos ya que los datos son reales y están buenos para practicar.

En el siguiente repositorio pueden encontrar el significado de cada columna de los datasets:
https://github.com/Datathon2021/Recomendador

Consigna:

- Dividir set en train y test. Tomar como train los datos hasta el 1 de marzo de 2021. Desde el 1ro de marzo en adelante, reservar para test.
- Desarrollar un recomendador. El recomendador debe ser capaz de generar recomendaciones para TODOS los usuarios (incluyendo los cold start que no tengan visualizaciones en el set de train). Generar 20 recomendaciones por usuario.
- Las recomendaciones tienen que ser para cada account_id y hay que recomendar content_id (NO asset_id). Pueden encontrar esto en el repositorio de la competencia.
- Los contenidos que recomienden, no tienen que haber sido vistos previamente por los usuarios (filtrar).
- Evaluarlo con MAP.

Recomendaciones:
- En este caso no tenemos ratings explícitos como los casos que vimos, deben generar ustedes estos ratings mediante algún criterio. Lo más simple podría ser utilizar ratings binarios (lo vió / no lo vió).
- Hay una columna que nos indica hasta cuando va a estar disponible el contenido
- La columna **end_vod_date**: "fecha de finalización de la disponibilidad del activo en la plataforma" puede llegar a serles muy útil. ¿Tiene sentido recomendar algo que no va a estar disponible en el set de test? (a partir del 1 de marzo de 2021).
- Comiencen con algo SIMPLE. No se compliquen con todas las columnas que tiene el dataset. No van a necesitar usar todas, muchas columnas podrán descartarlas dependiendo del enfoque que tomen.

Datos:
- Se encuentran adjuntos en la entrega

Fecha de entrega: 21/08/2024.

Pueden subirlo a un repositorio de github (público) y subir el link. De paso les sirve para ir armando su perfil de github con algunos proyectos 😉.



## train.csv
Este dataset contiene los registros de visualizaciones de contenidos de Flow del formato video on demand (VOD), correspondiente a una muestra aleatoria de más de 113 mil perfiles. A continuación, se detalla el diccionario de variables de esta tabla:

- **customer_id:** código de identificación de cada cliente de Flow (puede tener asociados uno o más account_id)
- **account_id:** código de identificación de cada perfil de Flow (se corresponde con un único customer_id)
- **device_type:** indica el tipo de dispositivo desde el que se efectuó la visualización. Las categorías posibles son:
   - **CLOUD:** cliente web
   - **PHONE:** teléfono celular
   - **STATIONARY:** smart TV
   - **STB:** set-top box, el decodificador Flow
   - **TABLET**
- **asset_id:** código de identificación de cada activo (video) disponible en la plataforma
- **tunein:** fecha y hora de inicio de cada visualización
- **tuneout:** fecha y hora de finalización de cada visualización
- **resume:** variable dummy que indica si se reanuda un consumo anterior del mismo asset_id

## metadata.csv
Contiene la metadata asociada a cada uno de los contenidos. Las variables incluidas son:

- **asset_id:** código de identificación de cada activo (video) disponible en Flow
- **content_id:** código de identificación que agrupa los distintos asset_id asociados a un mismo contenido (por ejemplo, cada episodio de una misma serie tiene su propio asset_id, mientras que la serie se identifica con un content_id único)
- **title:** título
- **reduced_title:** título reducido
- **episode_title:** título del episodio (válido para contenidos episódicos, como las series)
- **show_type:** tipo de show - las categorías son autorreferenciales con excepción de “Rolling”, que indica que se trata de una serie incompleta, y “Web”, la cual remite a contenidos pensados íntegramente en formato digital (series web) -
- **released_year:** año de lanzamiento
- **country_of_origin:** país de origen – expresado con el código de dos letras propio del estándar internacional de normalización ISO 3166 -
- **category:** categoría o género al que pertenece el contenido - puede haber una o más -
- **keywords:** palabras clave o tags asociadas al contenido - puede haber una o más -
- **description:** descripción (sinopsis)
- **reduced_desc:** descripción (sinopsis) reducida
- **cast_first_name:** nombre y apellido de los actores y actrices principales
- **credits_first_name:** nombre y apellido del director o directora
- **run_time_min:** duración total, expresada en minutos
- **audience:** audiencia target
- **made_for_tv:** variable dummy que indica si el contenido fue hecho para TV
- **close_caption:** variable dummy que indica si el contenido posee subtítulos
- **sex_rating:** variable dummy que indica si el contenido tiene escenas de sexo explícitas
- **violence_rating:** variable dummy que indica si el contenido tiene escenas de violencia explícitas
- **language_rating:** variable dummy que indica si el contenido posee lenguaje que puede ser considerado ofensivo o inapropiado
- **dialog_rating:** variable dummy que indica si el contenido posee diálogos que pueden ser considerado ofensivos o inapropiados
- **fv_rating:** variable dummy que indica si el contenido tiene rating de FV, que corresponde a público infantil con violencia ficticia
- **pay_per_view:** variable dummy que indica si se trata de un alquiler
- **pack_premium_1:** variable dummy que indica si se trata de un contenido exclusivo del pack premium 1
- **pack_premium_2:** variable dummy que indica si se trata de un contenido exclusivo del pack premium 2
- **create_date:** fecha de creación del activo
- **modify_date:** fecha de modificación del activo
- **start_vod_date:** fecha desde la cual el activo se encuentra disponible en la plataforma
- **end_vod_date:** fecha de finalización de la disponibilidad del activo en la plataforma

# Instalamos e importamos los paquetes necesarios para ejecutar el codigo

In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Subimos y leemos los archivos csv para realizar un vistazo de los datos que contiene

In [25]:
%cd /content

/content


In [26]:
!git --version

git version 2.34.1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [34]:
!git init
!git remote add origin https://github.com/Martinccv/Proyecto_final_DS.git
!git config core.sparsecheckout true
!git sparse-checkout set data.csv
!git pull origin main #puede ser master
!git checkout

hint: Using 'master' as the name for the initial branch. This default branch name
hint: is subject to change. To configure the initial branch name to use in all
hint: of your new repositories, which will suppress this warning, call:
hint: 
hint: 	git config --global init.defaultBranch <name>
hint: 
hint: Names commonly chosen instead of 'master' are 'main', 'trunk' and
hint: 'development'. The just-created branch can be renamed via this command:
hint: 
hint: 	git branch -m <name>
Initialized empty Git repository in /content/.git/
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 13 (delta 0), reused 7 (delta 0), pack-reused 0
Unpacking objects: 100% (13/13), 4.45 MiB | 4.90 MiB/s, done.
From https://github.com/Martinccv/Proyecto_final_DS
 * branch            main       -> FETCH_HEAD
 * [new branch]      main       -> origin/main


In [35]:
df_train=pd.read_csv('/content/data.csv/train1.csv')

In [36]:
df_train.head()

,customer_id,account_id,device_type,asset_id,tunein,tuneout,resume
0,0,90627,STATIONARY,18332.0,2021-02-18 22:52:00.0,2021-02-18 23:35:00.0,0
1,0,90627,STATIONARY,24727.0,2021-03-24 23:17:00.0,2021-03-25 00:01:00.0,0
2,1,3387,STB,895.0,2021-03-15 10:05:00.0,2021-03-15 10:23:00.0,0
3,1,3387,STB,895.0,2021-03-15 10:23:00.0,2021-03-15 11:18:00.0,1
4,1,3387,STB,26062.0,2021-03-16 09:24:00.0,2021-03-16 09:44:00.0,0


In [37]:
df_metadata=pd.read_csv('/content/data.csv/metadata.csv', sep=';')

In [38]:
df_metadata.head()

,asset_id,content_id,title,reduced_title,episode_title,show_type,released_year,country_of_origin,category,keywords,...,language_rating,dialog_rating,fv_rating,pay_per_view,pack_premium_1,pack_premium_2,create_date,modify_date,start_vod_date,end_vod_date
0,15188,0.0,Ep:17 Tiempos Compulsivos,Tiempos_Compul_E17,Episodio 17,Serie,2012.0,AR,Drama,"Trastornos,Médicos,Tragicómica,Telenovela,Enfe...",...,N,N,N,N,N,N,2017-12-01T10:18:15.0Z,2019-01-26T06:37:18.0Z,2017-12-01T00:00:00.0Z,2020-12-01T23:59:59.0Z
1,24940,1.0,7 Cajas,7_Cajas,NaN,Película,2012.0,PY,Suspenso/Acción,"Latinoamérica,Pobreza,Crimen,Pandillas",...,N,N,N,Y,N,N,2017-12-19T20:58:15.0Z,2019-09-17T19:02:03.0Z,2017-12-15T00:00:00.0Z,2022-12-14T23:59:59.0Z
2,21939,2.0,La Maldición de las Hormigas Gigantes,La_Maldicion_de_las,NaN,Película,2016.0,FI,Terror/Comedia,"Criaturas,Plagas,Adolescentes,Fantasía,Video J...",...,N,N,N,N,N,N,2018-02-16T13:51:07.0Z,2020-04-28T14:16:38.0Z,2018-01-25T00:00:00.0Z,2020-12-01T23:59:59.0Z
3,9005,3.0,Una Mujer Fantástica,Una_Mujer_Fantastic,NaN,Película,2017.0,CL,Drama,"LGBT,Mujeres,Latinoamérica",...,N,N,N,N,Y,N,2018-05-26T11:58:44.0Z,2019-11-15T03:00:23.0Z,2018-05-27T00:00:00.0Z,2021-04-30T23:59:59.0Z
4,7391,4.0,Star Trek,Star_Trek,NaN,Película,2009.0,US,Ciencia Ficción/Aventura,"Fantasía,Galaxia,Futurismo,Aliens,Criaturas",...,N,N,N,Y,N,N,2019-05-03T20:07:24.0Z,2020-04-09T04:37:29.0Z,2019-05-02T00:00:00.0Z,2020-12-31T23:59:59.0Z


In [39]:
df_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33144 entries, 0 to 33143
Data columns (total 30 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   asset_id            33144 non-null  int64  
 1   content_id          33123 non-null  float64
 2   title               33144 non-null  object 
 3   reduced_title       33144 non-null  object 
 4   episode_title       28997 non-null  object 
 5   show_type           33140 non-null  object 
 6   released_year       33144 non-null  float64
 7   country_of_origin   33140 non-null  object 
 8   category            33144 non-null  object 
 9   keywords            33142 non-null  object 
 10  description         33142 non-null  object 
 11  reduced_desc        33144 non-null  object 
 12  cast_first_name     24412 non-null  object 
 13  credits_first_name  20590 non-null  object 
 14  run_time_min        33144 non-null  float64
 15  audience            33143 non-null  object 
 16  made

In [40]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3657801 entries, 0 to 3657800
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   customer_id  int64  
 1   account_id   int64  
 2   device_type  object 
 3   asset_id     float64
 4   tunein       object 
 5   tuneout      object 
 6   resume       int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 195.3+ MB


In [ ]:
df_train.isna().sum()

,0
customer_id,0
account_id,0
device_type,29
asset_id,22
tunein,0
tuneout,0
resume,0


In [ ]:
df_metadata.isna().sum()

,0
asset_id,0
content_id,21
title,0
reduced_title,0
episode_title,4147
show_type,4
released_year,0
country_of_origin,4
category,0
keywords,2
